# Imports

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import RegressorChain
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor
from IPython import get_ipython
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Setup plots

In [3]:
plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

get_ipython().config.InlineBackend.figure_format = 'retina'

# Some functions to convert values in column between numbers and strings

In [4]:
dictionaries = {}
def convert_to_digits(df, col):
    uniques = df[col].unique()
    #index = df.index
    unique_names = {}
    digits = []
    i = 0
    for uniq in uniques:
        unique_names[uniq] = i
        i = i + 1
    for name in tqdm(df[col]):
        digits = digits + [unique_names[name]]
    dictionaries[col] = unique_names
    return digits

def get_key(d, value):
    return [key for key, val in d.items() if val==value]
    
def convert_back(df, col):
    unique_names = dictionaries[col]
    names = []
    for digit in tqdm(df[col]):
        names = names + get_key(unique_names, digit)
    return names

# Additional useful functions

In [5]:
def make_lags(ts, lags, lead_time=1, name='y'):
    return pd.concat(
        {
            f'{name}_lag_{i}': ts.shift(i)
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)

In [6]:
def make_leads(ts, leads, name='y'):
    return pd.concat(
        {f'{name}_lead_{i}': ts.shift(-i)
         for i in reversed(range(leads))},
        axis=1)

# Reading needed files with data

In [7]:
holidays_events = pd.read_csv(
    './store-sales-time-series-forecasting/holidays_events.csv',
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
holidays_events = holidays_events.set_index('date').to_period('D')

In [8]:
stores = pd.read_csv(
    './store-sales-time-series-forecasting/stores.csv',
    dtype={
        'store_nbr': 'category',
        'city': 'category',
        'state': 'category',
        'type': 'category',
        'cluster': 'category',
    },
    index_col = 'store_nbr',
)

In [9]:
stores['city'] = convert_to_digits(stores, col='city')
stores['state'] = convert_to_digits(stores, col='state')
stores['type'] = convert_to_digits(stores, col='type')

100%|██████████| 54/54 [00:00<00:00, 4915.31it/s]


In [10]:
transactions = pd.read_csv(
    './store-sales-time-series-forecasting/transactions.csv',
    parse_dates=['date'],
    infer_datetime_format=True,
)
transactions = transactions.set_index('date').to_period('D')

In [11]:
transactions

,store_nbr,transactions
date,,
2013-01-01,25,770
2013-01-02,1,2111
2013-01-02,2,2358
2013-01-02,3,3487
2013-01-02,4,1922
...,...,...
2017-08-15,50,2804
2017-08-15,51,1573
2017-08-15,52,2255


In [12]:
oil = pd.read_csv(
    './store-sales-time-series-forecasting/oil.csv',
    parse_dates=['date'],
    infer_datetime_format=True,
)
oil = oil.set_index('date').to_period('D')

In [14]:
store_sales = pd.read_csv(
    './store-sales-time-series-forecasting/train.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales['store_nbr_feature'] = store_sales['store_nbr'].copy()
store_sales['family_feature'] = convert_to_digits(store_sales, col='family')
store_sales['day'] = store_sales.index.day
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()
store_sales = store_sales.join(oil.fillna(method='bfill'), on='date')
store_sales = store_sales.join(stores, on='store_nbr')
store_sales = store_sales.join(transactions, on=['store_nbr', 'date'])
store_sales['dcoilwtico'] = store_sales['dcoilwtico'].fillna(method='bfill')

  5%|▌         | 153264/3000888 [00:43<13:20, 3559.32it/s]


KeyboardInterrupt: 

In [ ]:
store_sales

# Creating HybridModel class

In [262]:
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None
        
    def fit(self, X_1, X_2, y, alpha):
        self.model_1.fit(X_1, y)
        y_fit = pd.DataFrame(self.model_1.predict(X_1), index=X_1.index, columns=y.columns)
        y_resid = y - y_fit
        y_resid = y_resid.stack().squeeze()
        X_sr = create_sales_window_features(X_2, y_resid,alpha)
        X = create_onpromo_window_features(X_sr, alpha)
        self.model_2.fit(X, y_resid)
        self.y_columns = y.columns
        self.y_fit = y_fit
        self.y_resid = y_resid
        
    def predict(self, X_1, X_2):
        y_pred = pd.DataFrame(self.model_1.predict(X_1), index=X_1.index, columns=self.y_columns)
        y_pred = y_pred.stack().squeeze()
        y_pred += self.model_2.predict(X_2)
        return y_pred.unstack()
    
    def create_sales_window_features(self, X, y_resid, alpha):
        X_2 = X.copy()
        X_2.loc[:, 'sales'] = y_resid
        for col in tqdm(X_2.columns):
        if col[0] == 'sales':
            name_lags = ('sales_lags_1', col[1], col[2])
            name_mean = ('sales_lags_mean_7', col[1], col[2])
            name_std = ('sales_lags_std_7', col[1], col[2])
            new_cols = pd.MultiIndex.from_tuples([name_lags, name_mean, name_std])
            X_2 = X_2.reindex(columns=X_2.columns.union(new_cols))
            X_2[name_lags] = make_lags(X_2[col], lags=1)
            X_2[name_mean] = X_2[name_lags].ewm(alpha=alpha).mean()
            X_2[name_std] = X_2[name_lags].ewm(alpha=alpha).std()
        return X_2
    
    def create_onpromo_window_features(self, X, alpha):
        X_2 = X.copy()
        for col in tqdm(X_2.columns):
        if col[0] == 'onpromotion':
            onpromotion = X_2[col].squeeze().rename('onpromotion')
            name_lags = ('onpromotion_lags_1', col[1], col[2])
            name_leads = ('onpromotion_lead_0', col[1], col[2])
            name_mean = ('onpromotion_mean_7', col[1], col[2])
            new_cols = pd.MultiIndex.from_tuples([name_lags, name_leads, name_mean])
            X_2 = X_2.reindex(columns=X_2.columns.union(new_cols))
            X_2[name_lags] = make_lags(onpromotion, lags=1, name='y_promo')
            X_2[name_leads] = make_leads(onpromotion, leads=1)
            X_2[name_mean] = onpromotion.ewm(alpha=alpha).mean()
        return X_2

# Creating model for Store Sales - Time Series Forecasting competition

### Data processing

In [263]:
df_train = store_sales.unstack(['store_nbr', 'family']).loc["2017"]
y = df_train.loc[:, 'sales'].squeeze()

In [264]:
fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X_1 = dp.in_sample()
X_1['NewYear'] = (X_1.index.dayofyear == 1)

In [ ]:
X_2 = df_train.drop('sales', axis=1).stack()

In [ ]:
model = BoostedHybrid(model_1=LinearRegression(), model_2=XGBRegressor())

In [ ]:
y_train, y_valid = y[:"2017-07-01"], y["2017-07-02":]
X1_train, X1_valid = X_1[: "2017-07-01"], X_1["2017-07-02" :]
X2_train, X2_valid = X_2.loc[:"2017-07-01"], X_2.loc["2017-07-02":]

model.fit(X1_train, X2_train, y_train, 0.5)
y_fit = model.predict(X1_train, X2_train).clip(0.0)
y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_valid, y_pred) ** 0.5
print(f'Training RMSLE: {rmsle_train:.5f}')
print(f'Validation RMSLE: {rmsle_valid:.5f}')

families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(11, 9), **plot_params, alpha=0.5,
)
_ = y_fit.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C0', ax=axs)
_ = y_pred.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C3', ax=axs)
for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

In [ ]:
efficiency = {}
alphas = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
for alpha in alphas:
    model.fit(X1_train, X2_train, y_train, alpha)
    y_fit = model.predict(X1_train, X2_train).clip(0.0)
    y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

    rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
    rmsle_valid = mean_squared_log_error(y_valid, y_pred) ** 0.5
    
    efficiency[alpha] = rmsle_valid
    
    print(f'Training RMSLE: {rmsle_train:.5f}')
    print(f'Validation RMSLE: {rmsle_valid:.5f}')

min_val_error = min(efficiency, key=efficiency.get)
print(f'Minimal validation error is for alpha = {min_val_error}')

In [265]:
#model = LinearRegression(fit_intercept=False)
#model.fit(X_1, y)
#y_pred = pd.DataFrame(model.predict(X_1), index=X_1.index, columns=y.columns)

In [266]:
#y_deseason = y - y_pred

In [ ]:
#STORE_NBR = '1'  # 1 - 54
#FAMILY = 'PRODUCE'
# Uncomment to see a list of product families
# display(store_sales.index.get_level_values('family').unique())

#ax = df_train.loc(axis=1)['sales', STORE_NBR, FAMILY].plot(**plot_params)
#ax = y_pred.loc(axis=1)[STORE_NBR, FAMILY].plot(ax=ax)
#ax = y_deseason.loc(axis=1)[STORE_NBR, FAMILY].plot(ax=ax)
#ax.set_title(f'{FAMILY} Sales at Store {STORE_NBR}');

In [268]:
#df_train_deseason = df_train.copy()
#df_train_deseason.loc[:, 'sales'] = y_deseason

In [269]:
#df_train_deseason

In [270]:
#df_train_deseason = df_train.copy()
#for col in tqdm(df_train.columns):
#    if col[0] == 'sales':
#        df_train_deseason[col] = y_deseason[(col[1], col[2])]

In [275]:
df_train_final = df_train_promo.copy()

### Loading test data and check the efficincy of our model on it

In [276]:
df_test = pd.read_csv(
    './store-sales-time-series-forecasting/test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

# Create features for test set
X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'
X_test['NewYear'] = (X_test.index.dayofyear == 1)

y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])

In [277]:
y_submit = y_submit.drop(columns=['city','cluster','dcoilwtico','onpromotion','state','type'])
#y_submit.to_csv('submission.csv', index=False)

In [279]:
df_test

id  onpromotion
store_nbr family     date                            
1         AUTOMOTIVE 2017-08-16  3000888            0
                     2017-08-17  3002670            0
                     2017-08-18  3004452            0
                     2017-08-19  3006234            0
                     2017-08-20  3008016            0
...                                  ...          ...
9         SEAFOOD    2017-08-27  3022271            0
                     2017-08-28  3024053            0
                     2017-08-29  3025835            0
                     2017-08-30  3027617            0
                     2017-08-31  3029399            0

[28512 rows x 2 columns]

In [278]:
y_submit

date        store_nbr  family                    
2017-08-16  1          AUTOMOTIVE                       4.274030
                       BABY CARE                        0.000000
                       BEAUTY                           3.495530
                       BEVERAGES                     2414.541711
                       BOOKS                            0.438314
                                                        ...     
2017-08-31  9          POULTRY                        372.417153
                       PREPARED FOODS                 107.843530
                       PRODUCE                       1273.316507
                       SCHOOL AND OFFICE SUPPLIES      44.945937
                       SEAFOOD                         18.946715
Length: 28512, dtype: float64